# Mapping Dead Reckoning course data

Hello and welcome to this notebook. 

## Aims

There are three key aims of this notebook:
- Gain some understanding of the Folium plugin.
- Deal with code that isn't behaving as expected.
- Map some ship course data, including showing how we can convert data to multiple courses.  

## Data 

In the previous notebook, we downloaded some data from Discovery while investigating how to use the Discovery API. Here, we are going to use the images we downloaded - the ships logs from the 1800's. 

## Dead reckoning

Prior to the GPS system, or other satellite navigation systems, ships navigated using dead reckoning. This is a method of navigation that uses the last known position of the ship, and the course and speed of the ship, to estimate the current position. This means the log recorded the course and speed of the ship, and the time at which the course and speed were recorded. 

## Ships logs and heading data. 

In the 1800's, ships compasses were less accurate than modern 3-digit system compasses. As a consequence, they used a [32 wind compass rose](https://en.wikipedia.org/wiki/Points_of_the_compass#32-wind_compass_rose). If we investigate the logs we've downloaded, they will contain headings such as "ENE" or "NW". These are about as accurate as ships could navigate at this point, and represent a range of headings of 11.25 degrees, which we will need to compensate for. 
Helpfully, we can make an assumption about these ranges, making the amount of data we need to generate more manageable. As we have to assume that the data in the log are accurate, we can assume that the ship was anywhere within the range of headings - for example, for ENE, this would be 78.75 to 90 degrees. The most likely was that the ship followed the middle of this course, but that they were not precise, and that the most out-of-course option is the limit of where they could be. So, we only need to generate 3 options - the most likely, and the two extremes. 
So, rather than generating 1 data point for each heading, we will generate 3. At the end of the course, this will give a range of positions that the ship could be in.

### Sighting land

This technique isn't perfect; once the ship sighted land, they would have been able to take an accurate location. In reality, this would have fixed any drift from the dead reckoning. However, we are aiming to keep this notebook understandable, so for now we are going to assume that the ship didn't sight land until the end of the journey. 

### Entering the data

#### Creating the lists

The first step is to create the lists that we will use to store the data. We will create a list for the left-most, middle and right-most data points. 

In [ ]:
(code)

#### The first point

We need to start with known good location - this will provide the initial lat and long that the rest of the course is built from. So first, we need to ask the user for this information. 
When the user runs this cell, they will be asked for this lat and long. This will then be used as the first waypoint for each of the three lists.

In [ ]:
(code)

#### The next points

After the known good point, all we know is the bearing speed and time. 
So, we ask the user for this information, and use it to work out the next point. If the world was flat, this would be some simple trigonometry - we know the bearing, and we know the speed and time so we can work out the distance. We can then use these data to work out the lat and long of the next point. The world, however, isn't flat. Luckily, the [geopy](https://geopy.readthedocs.io/en/stable/) library has a function that will do this for us - geodesic. This takes the start, bearing, and distance, and returns the lat and long of the end point.
Here, we start to do some additional work - we also need to work out the left and right points. For this, we are going to take the bearing that the user provided, and add and subtract 5.625 degrees, and work out the left and right points.

In [ ]:
(code)

### Mapping the data

We are going to use the Folium plugin to map the data. This is a very powerful plugin, with a range of options. Here, we are going to use 2 of them:
- Polyline - this will draw a line between the points we give it.
- Marker - this will put a marker on the map at the point we give it, and allow us to add additional information. 

### Static maps

Folium is an additive tool; we can add multiple features as we desire. For this static map, we are going to start with two - a line, and a marker at each waypoint.

The line is going to use the PolyLine function, and will take a list of coordinates. We are going to draw a line for each of the lists, with the main one being bigger and bolder than the others.

For the points, we are going to use the Marker tool. 

We are going to start by creating an empty map. 
In this example we will create a world map by centring on 0,0 and using a zoom level of 2. If you know the limit of your data, feel free to experiment by changing these values. 

In [ ]:
(code)

Next, we are going to start adding the features. We will start with the main line. We start by creating a temporary list, isolating the coordinates from the main list. For this, we loop through the main list, selecting the lat and long each time. 

In [ ]:
(code)

 We then add this to the map using the PolyLine function. 

In [ ]:
(code)

We then do the same function, but for the left and right lists. 

In [ ]:
(code)

After this, we add the points. We don't need to create the temporary list this time, we can just cycle through the main list, extracting the data we need at each point. 

In [ ]:
(code)

Finally, we show the map again.

In [ ]:
(code, and a map!)

##### Other things we may want to add

If we investigate the Folium documentation, we can see there are a number of different features we may want to add to the map, depending on the information we have available, and what we want to show. In all the cases, the documentation shows what data are needed, and in what format. 
For example, the [boat marker](https://python-visualization.github.io/folium/latest/user_guide/plugins/boat_marker.html) takes a heading and location for the boat, a heading and speed for the wind, and colour for the marker. If these data are available in the raw data, these could be added to the original master list, then when drawing the map, cycle through them similar to how we did for the points.
If the mapped ships course covers the world, but has a lot of detail, we may want to add a [minimap](https://python-visualization.github.io/folium/plugins.html#folium.plugins.MiniMap) to the map. This will allow the user to see where the map is in relation to the world. In this case, we only need to do it once. We will need to provide the same information as the main map - the center and zoom level. We can also provide position relative to the main map. 

## Animating the data

As we have time and location, we can animate the data. This will give us a really nice way of visualising the data; it will give context on how fast the ship was travelling at any one point. 

Note: this is where the code isn't behaving as expected. The next few cells are for advanced users only, and can be skipped - if you want to see the animation, skip to the next section.

One of the main changes between polyline and timestampedgeojson is the data required. For polyline we just needed a list of coordinates, for timestampedgeojson we need start and end coordinates and start and end times.
We already have this data, we just need to do some shuffling to get it into the right arrangement.

In [ ]:
(code)

### How to animate the data

Note: this is where the code isn't behaving as expected. The next few cells are for advanced users only, and the details can be ignored - if you want to see the animation, and you're using the default data, run the cells in order and the final cell will show the animation.

Now we have the data in the right format arrangement, its a similar process to the static map. We start by creating an object called features - this will get filled by the data we want to animate. Then, similar to the static map, we create a map, and add the animation to it.

In [ ]:
(code, and a map that isn't animated)

### That didn't work, what happened?

As we can see, the animation didn't work. 
Typically, we would look at the error message and work out a solution from the information provided. However, here we just get the message "Time not available". 
So, we need to look on GitHub to see if anyone else has had this problem - the issues tab is a good place to start. Here, we find [this issue](https://github.com/socib/Leaflet.TimeDimension/issues/83), where another user has had the same problem. As part of the conversation on this issue, we find a link to [an issue on a different repository](https://github.com/socib/Leaflet.TimeDimension/pull/214), seemingly fixing the same issue. 

We can investigate further. We know that Folium uses the JavaScript library (Leaflet)[], and searching the web for "Leaflet TimeStampedGeoJson" leads us to [this page](https://github.com/python-visualization/folium/blob/main/folium/plugins/timestamped_geo_json.py) showing the code for the plugin. From here, at [line 77](https://github.com/python-visualization/folium/blob/main/folium/plugins/timestamped_geo_json.py#L77), we can find our way to the original plugin code, [here](https://github.com/socib/Leaflet.TimeDimension). 

This is good news, as we have confirmed that the issue is in the plugin, and not in our code. We can also see that the plugin is still being maintained, and that the issue has been fixed. All that we're wating for is a new release of the plugin, and for Leaflet and Folium to update to the new version.

For us, this leaves us with two options:
- Wait for the new version to be released, and update our code when it is.
- We know that the issue is with dates before 1970, so we can change the dates we're working with. For this example, we're sure the dates we're working with are all in the 1800's, so we can add 200 years to all of the dates. 

#### Adding 200 years to the dates
Adding 200 years to the date is a simple process, and we can do this using the datetime library. We simply loop through the dates, adding 200 years to each one.

In [ ]:
(code)

### Re-running the animation

Now that we have updated the dates to post 1970, we can re-run the animation.

In [ ]:
(code)